In [1]:
from datasets import load_dataset, DatasetDict, concatenate_datasets
import pandas as pd
from datasets import get_dataset_config_names, get_dataset_split_names
import duckdb


pd.set_option("display.max_colwidth", None)


In [2]:
# !pip install -U datasets clean-text jusText nltk numpy beautifulsoup4 lxml
from datasets import load_dataset
import numpy as np
from cleantext import clean
import os, nltk
NLTK_USER_DIR = os.path.expanduser("~/nltk_data")


nltk.download("punkt", quiet=True)

import os, nltk, re
from bs4 import BeautifulSoup
import hashlib

# Make sure NLTK looks in your user directory
NLTK_USER_DIR = os.path.expanduser("~/nltk_data")
if NLTK_USER_DIR not in nltk.data.path:
    nltk.data.path.append(NLTK_USER_DIR)

def ensure_punkt():
    """
    Ensure Portuguese punkt is available and visible to this kernel.
    Newer NLTK may also require 'punkt_tab'.
    """
    try:
        nltk.data.find("tokenizers/punkt/portuguese.pickle")
    except LookupError:
        # download into the same directory we added to nltk.data.path
        nltk.download("punkt", download_dir=NLTK_USER_DIR, quiet=True)
        try:
            nltk.data.find("tokenizers/punkt/portuguese.pickle")
        except LookupError:
            nltk.download("punkt_tab", download_dir=NLTK_USER_DIR, quiet=True)
            nltk.data.find("tokenizers/punkt/portuguese.pickle")

# ----------------------------
# Author's regex & helpers (verbatim)
# ----------------------------
import re
from bs4 import BeautifulSoup
import hashlib

HTML_RE = re.compile(r"<[^>]+>")
URL_RE = re.compile(r"((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#…])*")
HASHTAG_RE = re.compile(r"#(\w+)")
QUOTE_SPACE_START_RE = re.compile(r"^\"\s")
QUOTE_SPACE_END_RE = re.compile(r"\s\"$")
MENTION_RE = re.compile(r"@(\w+)")
RETWEET_RE = re.compile(r"RT @(\w+):")
COD_RE = re.compile(r"COD _ (\w+) ")
BULLET_RE = re.compile(r"^(\d)+.\s")
THREE_DASH_RE = re.compile(r"---.*---")
MORE_THAN_THREE_POINTS_RE = re.compile(r"\.{4,}")

VALID_CHARS = "0123456789abcdefghijklmnopqrstuvwxyzàáâãåāèéêëěėēîïíìįīĵłñńôöòóōõšśûüùúūÿýźçćčñń!\"#$%&'()*+,-./:;<=>?@[\\]^_`{|}~«»“”ºª€ \t\n\r\x0b\x0c"

INVALID_START = [
    "List of recent changes","Sort by","Home |","> Home","useful tips","Licenses:","Search in: ",
    "Terms of Use - ","Home page","Home Page","Copyright","Results/Page",
    "!","#","$","%","&","*","+",
    ",","-",".","/",":",";","<","=",
    ">","?","@","[","\\","]","^","_","`","{","|","}","~",
]
INVALID_MIDDLE = [" @ ", " / ", " | ", "[...]", "(...)"]
INVALID_END = [" ("]

MONTHS = ["january","february","march","april","may","june","july","august","september","october","november","december"]
SPEAKER_LABEL_SINGLE_LETTER = re.compile(
    r'(?m)(^|\s|[(\["“])'          # \1 = boundary we preserve
    r'[A-Za-zÀ-ÖØ-öø-ÿ]\.'         # exactly one letter + dot
    r'\s*(?:--|[-–—]{1,2})\s*'     # "--" or one/two dashes (hyphen/en/em), with spaces
)

# Word followed by double hyphen (speaker label), preserving the boundary before it
SPEAKER_LABEL_WORD_DASH = re.compile(
    r'(?m)(^|\s)[A-Za-zÀ-ÖØ-öø-ÿ]+(?:\.)?\s+--\s*'
)

# Dialogue-leading double dash right after strong punctuation or line start
DIALOGUE_LEADING_DASH = re.compile(
    r'(?m)(^|[\.!\?\:\;…])\s*--\s*'
)

# Capitalize the first letter after strong punctuation or at start of text/line.
# Keeps any opening quotes/brackets just before the letter.
CAP_SENT_START = re.compile(
    r'(?m)(^|[\.!\?\:\;…]\s+)([\"\'“”«»\(\[\{]*)([a-zà-öø-ÿ])'
)


def remove_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def remove_hashtags(text): return HASHTAG_RE.sub("", text).strip()
def remove_mentions(text): return MENTION_RE.sub("", text).strip()
def remove_retweets(text): return RETWEET_RE.sub("", text).strip()
def remove_urls(text): return URL_RE.sub("", text).strip()
def remove_cod_literature(text): return COD_RE.sub("", text).strip()
def remove_bullets(text): return BULLET_RE.sub("", text).strip()
def remove_three_dashes(text): return THREE_DASH_RE.sub("", text).strip()
def remove_quote_space_start(text): return QUOTE_SPACE_START_RE.sub('"', text)
def remove_quote_space_end(text):
    if text.endswith(' "'): return text[:-2] + '"'
    return text

def has_more_than_three_points(text): return bool(MORE_THAN_THREE_POINTS_RE.search(text))
def starts_with_month(text): return text.lower().startswith(tuple(MONTHS))
def has_too_long_word(text): return any(word for word in text.split(" ") if len(word) > 20)
def has_invalid_start(text): return text.startswith(tuple(INVALID_START))
def has_invalid_middle(text): return any(True for word in INVALID_MIDDLE if word in text)
def has_invalid_end(text): return text.endswith(tuple(INVALID_END))
def has_valid_brackets(text):
    return (text.count("(") == text.count(")") and text.count("[") == text.count("]") and text.count("{") == text.count("}"))
def has_valid_quotes(text): return text.count('"') % 2 == 0 and text.count("“") == text.count("”")
def is_empty(text): return len(text) == 0
def has_invalid_character(text):
    for char in text:
        if char.lower() not in VALID_CHARS: return True
    return False

def normalize_double_quotes(text: str) -> str:
    """
    Map guillemets/curly/double-low quotes to a plain double quote.
    Keeps the quotes; just standardizes the glyph.
    """
    if not text:
        return text
    return (text
            .replace("«", '"').replace("»", '"')   # guillemets
            .replace("“", '"').replace("”", '"')   # curly
            .replace("„", '"'))                    # low double

def remove_single_letter_speaker_labels(text: str) -> str:
    if not text:
        return text
    return SPEAKER_LABEL_SINGLE_LETTER.sub(r"\1", text)

def remove_speaker_label_word_dash(text: str) -> str:
    """
    Remove 'Word -- ' at line start or mid-sentence (e.g., 'Montoro -- '),
    keeping the whitespace/boundary that came before it.
    """
    if not text:
        return text
    return SPEAKER_LABEL_WORD_DASH.sub(r'\1', text)

def remove_dialogue_leading_double_dash(text: str) -> str:
    """
    Remove a dialogue-leading '--' when it appears right after strong punctuation
    (., !, ?, …, ;, :) or at the beginning of a line/text.
    Example: '... abstrata. -- Tinha sonhado...' -> '... abstrata. Tinha sonhado...'
    """
    if not text:
        return text
    return DIALOGUE_LEADING_DASH.sub(lambda m: (m.group(1) or '') + ' ', text)

def capitalize_sentence_starts(text: str) -> str:
    """
    Ensure sentences start with an uppercase letter at beginning or after
    strong punctuation (. ! ? … ; :). Preserves opening quotes/brackets
    immediately before the first letter.
    """
    if not text:
        return text

    # First, handle very start-of-text capitalization if needed
    # (also respects opening quotes/brackets)
    def _cap_at_start(s: str) -> str:
        m = re.match(r'^([\"\'“”«»\(\[\{]*)([a-zà-öø-ÿ])', s)
        if m:
            return m.group(1) + m.group(2).upper() + s[m.end():]
        return s

    text = _cap_at_start(text)

    # Then capitalize after strong punctuation + whitespace
    def repl(m):
        return (m.group(1) or '') + (m.group(2) or '') + m.group(3).upper()

    return CAP_SENT_START.sub(repl, text)


def author_transform_chain(text: str) -> str:
    text = remove_retweets(text)
    text = remove_mentions(text)
    text = remove_hashtags(text)
    text = remove_urls(text)
    text = remove_html_tags(text)
    text = normalize_double_quotes(text)
    text = remove_single_letter_speaker_labels(text)
    text = remove_speaker_label_word_dash(text)
    text = remove_dialogue_leading_double_dash(text)
    text = remove_cod_literature(text)
    text = remove_bullets(text)
    text = remove_three_dashes(text)
    text = remove_quote_space_start(text)
    text = remove_quote_space_end(text)

    # collapse multi-space
    text = re.sub(r'\s{2,}', ' ', text).strip()

    text = capitalize_sentence_starts(text)

    return text


# ----------------------------
# Paper-faithful helpers
# ----------------------------
def drop_nans_and_empties(ds):
    return ds.filter(lambda x: x["text"] is not None and len(x["text"].strip()) > 0)

# MEMORY-SAFE exact de-dup (streaming via hash set; single process to keep state)
def drop_exact_duplicates(ds, batch_size: int = 1000):
    seen = set()
    def tag_batch(batch):
        keep = []
        for t in batch["text"]:
            h = hashlib.md5(t.encode("utf-8")).hexdigest()
            if h in seen:
                keep.append(False)
            else:
                seen.add(h)
                keep.append(True)
        return {"__keep__": keep}
    ds = ds.map(tag_batch, batched=True, batch_size=batch_size, num_proc=1)
    ds = ds.filter(lambda k: k, input_columns="__keep__")
    ds = ds.remove_columns(["__keep__"])
    return ds

def apply_clean_text_ascii(s: str) -> str:
    return clean(
        s, fix_unicode=True, to_ascii=True, lower=False,
        no_line_breaks=False, no_urls=False, no_emails=False, no_phone_numbers=False,
        no_numbers=False, no_digits=False, no_currency_symbols=False,
    )

# Regex fallback if punkt still isn't available (keeps the pipeline running)
_FALLBACK_TOKEN_RE = re.compile(r"\w+|[^\w\s]")

def pt_word_count(s: str) -> int:
    try:
        ensure_punkt()
        return len(nltk.word_tokenize(s, language="portuguese"))
    except LookupError:
        # last-resort fallback (not paper-perfect, but close)
        return len(_FALLBACK_TOKEN_RE.findall(s))


def add_length_column(ds, batch_size: int = 1000):
    ensure_punkt()
    def _lens(batch):
        return {"__len__": [pt_word_count(t) for t in batch["text"]]}
    return ds.map(_lens, batched=True, batch_size=batch_size, num_proc=1)


def iqr_bounds_from_lengths(lengths):
    q1, q3 = np.percentile(lengths, 25), np.percentile(lengths, 75)
    iqr = q3 - q1
    return (q1 - 1.5*iqr, q3 + 1.5*iqr)

def apply_iqr_filter_on_cached_lengths(ds, lo, hi):
    ds = ds.filter(lambda L: lo <= L <= hi, input_columns="__len__")
    return ds.remove_columns(["__len__"])

# jusText boilerplate removal (paper: only for Web)
def web_justext(text: str) -> str:
    import justext
    paras = justext.justext(text, justext.get_stoplist("Portuguese"))
    good = [p.text for p in paras if p.class_type == "good"]
    return "\n".join(good) if good else text

def apply_clean_text_unicode_only(s: str) -> str:
    # Keep accents; still fix bad unicode
    from cleantext import clean
    return clean(
        s,
        fix_unicode=True,
        to_ascii=False,   # <-- keep accents
        lower=False,
        no_line_breaks=False,
        no_urls=False, no_emails=False, no_phone_numbers=False,
        no_numbers=False, no_digits=False, no_currency_symbols=False,
    )

# ----------------------------
# Integrated pipeline
# ----------------------------
def clean_one_domain_split(
    domain: str,
    split: str,
    use_author_transforms: bool = True,
    run_web_justext: bool = True,
    apply_author_filters: bool = True,
    keep_accents: bool = True,
    num_proc: int = 1,
    batch_size: int = 1000,
):
    """
    Paper pipeline + optional author steps, memory-safe:
      1) drop NaNs/empties
      2) jusText for Web (paper)
      3) author's transform chain (URLs/HTML/etc.)
      4) clean-text -> ASCII (paper)
      5) de-dup (streaming, hash-based)
      6) optional author filters (no tokenizer)
      7) add cached token lengths (NLTK), compute IQR, filter, drop length col (paper)
    """
    ds = load_dataset("liaad/PtBrVId-Raw", domain, split=split)
    # ds = ds[100]
    ds = drop_nans_and_empties(ds)

    if run_web_justext and domain == "web":
        ds = ds.map(lambda x: {"text": web_justext(x["text"])}, num_proc=num_proc, batched=False)

    # 3) author's transform chain (pure text cleanup)
    if use_author_transforms:
        ds = ds.map(lambda x: {"text": author_transform_chain(x["text"])}, num_proc=num_proc, batched=False)

    # 4) clean-text step — choose whether to preserve accents
    if keep_accents:
        ds = ds.map(lambda x: {"text": apply_clean_text_unicode_only(x["text"])}, num_proc=num_proc, batched=False)
    else:
        ds = ds.map(lambda x: {"text": apply_clean_text_ascii(x["text"])}, num_proc=num_proc, batched=False)
    ds = drop_exact_duplicates(ds, batch_size=batch_size)

    if apply_author_filters:
        ds = ds.filter(
            lambda t: (not starts_with_month(t))
                      and (not has_too_long_word(t))
                      and (not has_invalid_start(t))
                      and (not has_invalid_middle(t))
                      and (not has_invalid_end(t))
                      and (not has_more_than_three_points(t))
                      and (not is_empty(t))
                      and (not has_invalid_character(t))
                      and has_valid_brackets(t)
                      and has_valid_quotes(t),
            input_columns="text",
            num_proc=num_proc
        )

    # Cache token lengths once, then IQR-filter
    ds = add_length_column(ds, batch_size=batch_size)
    lo, hi = iqr_bounds_from_lengths(ds["__len__"])
    ds = apply_iqr_filter_on_cached_lengths(ds, lo, hi)

    return ds


Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


In [3]:
import os, nltk, pathlib

# 1) Pick a single directory and make NLTK look there
NLTK_USER_DIR = os.path.expanduser("~/nltk_data")
os.environ["NLTK_DATA"] = NLTK_USER_DIR  # ensure child processes see it too
if NLTK_USER_DIR not in nltk.data.path:
    nltk.data.path.insert(0, NLTK_USER_DIR)

# 2) Force-reinstall punkt + punkt_tab into that directory
nltk.download("punkt", download_dir=NLTK_USER_DIR, force=True, quiet=False)
# Newer NLTK also needs this metadata package
try:
    nltk.download("punkt_tab", download_dir=NLTK_USER_DIR, force=True, quiet=False)
except Exception:
    pass  # older NLTK won't have it

# 3) Verify both the PACKAGE and the Portuguese model are visible
print("NLTK paths:", nltk.data.path)
print("Has package dir? ", pathlib.Path(NLTK_USER_DIR, "tokenizers", "punkt").exists())
nltk.data.find("tokenizers/punkt")                       # should NOT raise
nltk.data.find("tokenizers/punkt/portuguese.pickle")     # should NOT raise

# Quick smoke test
from nltk import word_tokenize
print(word_tokenize("Olá mundo! Isto é um teste.", language="portuguese"))



[nltk_data] Downloading package punkt to
[nltk_data]     /home/laiarodrigo/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/laiarodrigo/nltk_data...


NLTK paths: ['/home/laiarodrigo/nltk_data', '/home/laiarodrigo/repos/Thesis/thesis/nltk_data', '/home/laiarodrigo/repos/Thesis/thesis/share/nltk_data', '/home/laiarodrigo/repos/Thesis/thesis/lib/nltk_data', '/usr/share/nltk_data', '/usr/local/share/nltk_data', '/usr/lib/nltk_data', '/usr/local/lib/nltk_data']
Has package dir?  True
['Olá', 'mundo', '!', 'Isto', 'é', 'um', 'teste', '.']


[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [4]:
from datasets import load_dataset, get_dataset_config_names, get_dataset_split_names
import random

def clean_one_domain_split_sample(
    domain: str,
    split: str,
    *,
    sample_size: int = 500,   # only clean this many per split
    seed: int = 123,
    use_author_transforms: bool = True,
    run_web_justext: bool = True,
    apply_author_filters: bool = True,
    keep_accents: bool = True,
    num_proc: int = 1,
    batch_size: int = 1000,
):
    ds_full = load_dataset("liaad/PtBrVId-Raw", domain, split=split)
    n = min(sample_size, len(ds_full))
    if n == 0:
        return ds_full

    # sample BEFORE heavy transforms
    rng = random.Random(seed)
    idxs = rng.sample(range(len(ds_full)), k=n)
    ds = ds_full.select(idxs)

    # now run the *same* cleaning steps, but only on the sample
    ds = drop_nans_and_empties(ds)

    if run_web_justext and domain == "web":
        ds = ds.map(lambda x: {"text": web_justext(x["text"])},
                    num_proc=num_proc, batched=False)

    if use_author_transforms:
        ds = ds.map(lambda x: {"text": author_transform_chain(x["text"])},
                    num_proc=num_proc, batched=False)

    if keep_accents:
        ds = ds.map(lambda x: {"text": apply_clean_text_unicode_only(x["text"])},
                    num_proc=num_proc, batched=False)
    else:
        ds = ds.map(lambda x: {"text": apply_clean_text_ascii(x["text"])},
                    num_proc=num_proc, batched=False)

    ds = drop_exact_duplicates(ds, batch_size=batch_size)

    if apply_author_filters:
        ds = ds.filter(
            lambda t: (not starts_with_month(t))
                      and (not has_too_long_word(t))
                      and (not has_invalid_start(t))
                      and (not has_invalid_middle(t))
                      and (not has_invalid_end(t))
                      and (not has_more_than_three_points(t))
                      and (not is_empty(t))
                      and (not has_invalid_character(t))
                      and has_valid_brackets(t)
                      and has_valid_quotes(t),
            input_columns="text",
            num_proc=num_proc
        )

    ds = add_length_column(ds, batch_size=batch_size)
    lo, hi = iqr_bounds_from_lengths(ds["__len__"])
    ds = apply_iqr_filter_on_cached_lengths(ds, lo, hi)
    return ds


def preview_ptbrvarid_samples_fast(
    n_per_split: int = 3,
    sample_size: int = 500,   # how many to clean per split before sampling to print
    *,
    keep_accents: bool = True,
    num_proc: int = 1,
    batch_size: int = 1000,
    seed: int = 123,
):
    def _short(s, n=400):
        s = (s or "").replace("\n", " ").strip()
        return (s[:n] + "…") if len(s) > n else s

    domains = get_dataset_config_names("liaad/PtBrVId-Raw")
    rng = random.Random(seed)

    for domain in domains:
        splits = get_dataset_split_names("liaad/PtBrVId-Raw", domain)
        print("\n" + "="*80)
        print(f"DOMAIN: {domain}")
        print("="*80)
        for split in splits:
            ds = clean_one_domain_split_sample(
                domain, split,
                sample_size=sample_size,
                seed=seed,
                keep_accents=keep_accents,
                num_proc=num_proc,
                batch_size=batch_size,
            )
            print(f"\n— {domain}/{split}: cleaned sample size = {len(ds):,}")
            if len(ds) == 0:
                print("  (no items after cleaning)")
                continue

            k = min(n_per_split, len(ds))
            idxs = rng.sample(range(len(ds)), k=k)
            for j, i in enumerate(idxs, 1):
                print(f"  [{j}/{k}] label={ds['label'][i]} | {_short(ds['text'][i])}")


preview_ptbrvarid_samples_fast(n_per_split=10, sample_size=300, keep_accents=True)



DOMAIN: journalistic


Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Filter:   0%|          | 0/300 [00:00<?, ? examples/s]

...

Filter:   0%|          | 0/300 [00:00<?, ? examples/s]

Filter:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/59 [00:00<?, ? examples/s]

Filter:   0%|          | 0/59 [00:00<?, ? examples/s]


— web/train: cleaned sample size = 52
  [1/10] label=1 | O Pavilhão de Desportos e Congressos de Matosinhos recebe a 9 de abril o encontro "GAM'EET", "dedicado 100% aos aficionadas de videojogos onde o público poderá gratuitamente jogar e conhecer alguns dos melhores gamers e streamers nacionais", sublinham os promotores. A iniciativa, que decorre entre as 10 e as 20 horas, terá disponíveis 32 posições de jogos, nos quais os visitantes poderão jogar com…
  [2/10] label=0 | Mulheres atendidas pelo Ceam de Japeri recebem cestas básicas graças à parceria entre Prefeitura e Governo do Estado Ação aconteceu nesta terça-feira (30) e contemplou cerca de 50 usuárias do equipamento. Inaugurado há menos de um mês, órgão oferece gratuitamente atendimento social, psicológico e orientação jurídica a cidadãs em situação de vulnerabilidade social
  [3/10] label=0 | Arquivos poá educação reforma escolas acessibilidade manutenção ensino estrutura - Prefeitura Municipal de Poá
  [4/10] label=0 | Você e

In [5]:
# # If needed: from datasets import load_dataset, concatenate_datasets
# from cleantext import clean
# import random, html

# # ASCII ONLY FOR THE MATCHING KEY (final text stays accented)
# def to_ascii_like_paper(s: str) -> str:
#     return clean(
#         s, fix_unicode=True, to_ascii=True, lower=False,
#         no_line_breaks=False, no_urls=False, no_emails=False, no_phone_numbers=False,
#         no_numbers=False, no_digits=False, no_currency_symbols=False,
#     )

# def ascii_key(s: str) -> str:
#     s = html.unescape(s).replace("\xa0", " ")
#     return " ".join(to_ascii_like_paper(s).split())

# def combine_splits_to_one(ds_dict, add_split_col=True):
#     from datasets import concatenate_datasets
#     parts = []
#     for split_name, split_ds in ds_dict.items():
#         if add_split_col:
#             split_ds = split_ds.map(lambda x, sn=split_name: {"__split__": sn})
#         parts.append(split_ds)
#     return concatenate_datasets(parts)

# def build_needed_keys_for_sample(official_all, idxs, text_col="text", label_col="label"):
#     keys_in_order = []
#     for i in idxs:
#         keys_in_order.append(f"{ascii_key(official_all[text_col][i])}||{official_all[label_col][i]}")
#     return keys_in_order, set(keys_in_order)

# def build_mapping_from_cleaned_sharded(clean_ds, needed_keys,
#                                        text_col="text", label_col="label",
#                                        num_shards=100, batch_size=500):
#     """Scan your preprocessed RAW (with accents) in shards; keep first hit for each needed key."""
#     mapping, remaining = {}, set(needed_keys)
#     for shard_idx in range(num_shards):
#         if not remaining:
#             break
#         shard = clean_ds.shard(num_shards=num_shards, index=shard_idx)
#         # compute combined keys for this shard in small batches
#         def _mk(batch):
#             cks = []
#             for t, lbl in zip(batch[text_col], batch[label_col]):
#                 cks.append(f"{ascii_key(t)}||{lbl}")
#             return {"__ckey__": cks}
#         shard_k = shard.map(_mk, batched=True, batch_size=batch_size, num_proc=1)
#         for i in range(len(shard_k)):
#             ck = shard_k["__ckey__"][i]
#             if ck in remaining:
#                 mapping[ck] = shard[text_col][i]   # ACCENTED, preprocessed text
#                 remaining.remove(ck)
#                 if not remaining:
#                     break
#         del shard, shard_k
#     return mapping

# from datasets import load_dataset

# DOMAIN   = "journalistic"
# N_SAMPLES = 20
# SEED      = 123

# # 1) Load official (paper) dataset and flatten splits
# official_dd  = load_dataset("liaad/PtBrVId", DOMAIN)   # ASCII text
# official_all = combine_splits_to_one(official_dd, add_split_col=True)

# # 2) Ensure you already have your preprocessed RAW (with accents)
# # If not already created earlier:
# # clean_ds = clean_one_domain(domain=DOMAIN, keep_accents=True, num_proc=1, batch_size=1000)

# # 3) Pick random official rows to compare
# import random
# random.seed(SEED)
# sample_idxs = random.sample(range(len(official_all)), k=min(N_SAMPLES, len(official_all)))

# # 4) Build keys just for those samples and find their matches inside your cleaned set
# keys_in_order, needed = build_needed_keys_for_sample(official_all, sample_idxs)
# raw_map = build_mapping_from_cleaned_sharded(clean_ds, needed, num_shards=100, batch_size=500)

# # 5) Pretty print comparisons
# def has_accents(s): return any(ord(c) > 127 for c in s)

# hits = 0
# print(f"\nComparing {len(sample_idxs)} random examples from '{DOMAIN}':\n")
# for i, ck in zip(sample_idxs, keys_in_order):
#     off_txt = official_all["text"][i]      # ASCII (paper release)
#     lbl     = official_all["label"][i]
#     split   = official_all["__split__"][i]
#     acc_txt = raw_map.get(ck)

#     print("—"*100)
#     print(f"[{split}] label={lbl}  |  idx={i}")
#     print("OFFICIAL (ASCII):")
#     print(off_txt[:600].replace("\n"," "))
#     if acc_txt is not None:
#         hits += 1
#         print("\nCLEANED RAW (ACCENTED):")
#         print(acc_txt[:600].replace("\n"," "))
#         print(f"\nHas accents? {has_accents(acc_txt)}")
#     else:
#         print("\nCLEANED RAW (ACCENTED): <not found — your pipeline may have filtered it out>")
#         print(f"Has accents? {has_accents(off_txt)}")

# print("\nSummary:")
# print(f"  Requested samples: {len(sample_idxs)}")
# print(f"  Found accented matches: {hits}")
# print(f"  Misses: {len(sample_idxs) - hits}")


este dataset nao é paralelo e apenas tem duas colunas: texto e label.  Primeiro, removeu entradas nulas, vazias e duplicadas. Em seguida, utilizou a biblioteca clean-text para corrigir erros de Unicode e normalizar todo o texto para ASCII. No domínio Web, aplicou ainda o jusText para eliminar sentenças irrelevantes e código-base HTML. Por fim, identificou e removeu outliers em cada domínio: calculou o intervalo inter-quartil (IQR) do número de tokens, usando o tokenizer do NLTK para português, e descartou textos com comprimento inferior a 
𝑄
1
−
1,5
×
IQR
Q1−1,5×IQR ou superior a 
𝑄
3
+
1,5
×
IQR
Q3+1,5×IQR. Este processo assegurou a exclusão de documentos demasiado curtos ou longos para o respetivo domínio, resultando num corpus mais limpo e consistente.

**//PUT ALL PROCESSED DOMAINS AND SPLITS INTO PTBRVARID TABLE AND THEN TO TRAIN AND TEST DATA TABLES**

In [6]:
# populate_ptbrvarid_full_fixed.py
# ------------------------------------------------------------
# Fully (re)populate DuckDB table `ptbrvarid` from PtBrVId-Raw.
# Requires: clean_one_domain_split(domain, split, ...)
# ------------------------------------------------------------

from datasets import get_dataset_config_names, get_dataset_split_names
import duckdb
import pandas as pd

# ----- label helpers & schema -----

def _label_to_name(ds, val):
    """Map int/str label to a readable name using HF features when available."""
    if isinstance(val, str):
        return val
    try:
        names = ds.features["label"].names
        if isinstance(val, int) and 0 <= val < len(names):
            return names[val]
    except Exception:
        pass
    return str(val)

def _is_pt_br(lbl_name: str, raw_label) -> bool:
    """
    Decide BR vs PT by normalized name; fallback to your rule:
    label 0 == pt-PT (European) → everything else BR.
    """
    s = (lbl_name or "").strip().lower().replace("_", "-")
    if s in {"pt-br", "br", "ptbr", "brazil", "brazilian"}:
        return True
    if s in {"pt-pt", "pt", "eu", "european"}:
        return False
    return (isinstance(raw_label, int) and raw_label != 0)

def _ensure_ptbrvarid_schema(con: duckdb.DuckDBPyConnection):
    # Create table if missing (6-column layout)
    con.execute("""
        CREATE TABLE IF NOT EXISTS ptbrvarid (
            dataset     TEXT,
            domain      TEXT,
            split       TEXT,
            label       TEXT,
            text_pt_br  TEXT,
            text_pt_pt  TEXT
        )
    """)
    # If the table already existed without `domain`, add it.
    cols = [r[1] for r in con.execute("PRAGMA table_info('ptbrvarid')").fetchall()]
    if "domain" not in cols:
        con.execute("ALTER TABLE ptbrvarid ADD COLUMN domain TEXT")
        print("[MIGRATE] Added missing column: domain")

# ----- main ingest -----

def ingest_ptbrvarid_all_to_duckdb(
    db_path: str,
    *,
    keep_accents: bool = True,
    num_proc: int = 1,
    batch_size_hf: int = 1000,
    batch_size_db: int = 5000,
):
    """
    Clears ptbrvarid and fully repopulates it with all cleaned rows.
    Prints progress per split, per domain, and final totals.
    """
    domains = get_dataset_config_names("liaad/PtBrVId-Raw")

    with duckdb.connect(db_path) as con:
        _ensure_ptbrvarid_schema(con)

        # wipe existing contents so this is the ONLY data in the table
        try:
            existing = con.execute("SELECT COUNT(*) FROM ptbrvarid").fetchone()[0]
        except duckdb.CatalogException:
            existing = 0
        con.execute("DELETE FROM ptbrvarid")
        print(f"[RESET] Cleared {existing} old rows from ptbrvarid")

        grand_total = 0

        for domain in domains:
            splits = get_dataset_split_names("liaad/PtBrVId-Raw", domain)
            domain_total = 0
            print(f"\n[DOMAIN] {domain} – {len(splits)} splits")

            for split in splits:
                print(f"  [CLEAN] {domain}/{split} …")
                ds = clean_one_domain_split(
                    domain, split,
                    keep_accents=keep_accents,
                    num_proc=num_proc,
                    batch_size=batch_size_hf,
                )

                buf, n = [], 0
                for ex in ds.to_iterable_dataset():
                    lbl_name = _label_to_name(ds, ex["label"])
                    text = ex["text"]
                    buf.append({
                        "dataset": "PtBrVarId",
                        "domain": domain,
                        "split": split,
                        "label": lbl_name,
                        "text_pt_br": text if _is_pt_br(lbl_name, ex["label"]) else None,
                        "text_pt_pt": text if not _is_pt_br(lbl_name, ex["label"]) else None,
                    })

                    if len(buf) >= batch_size_db:
                        df = pd.DataFrame(buf)
                        con.register("ptbr_buf", df)
                        # INSERT with explicit column list to avoid order mismatches
                        con.execute("""
                            INSERT INTO ptbrvarid (dataset, domain, split, label, text_pt_br, text_pt_pt)
                            SELECT dataset, domain, split, label, text_pt_br, text_pt_pt FROM ptbr_buf
                        """)
                        con.unregister("ptbr_buf")
                        n += len(buf)
                        buf = []

                if buf:
                    df = pd.DataFrame(buf)
                    con.register("ptbr_buf", df)
                    con.execute("""
                        INSERT INTO ptbrvarid (dataset, domain, split, label, text_pt_br, text_pt_pt)
                        SELECT dataset, domain, split, label, text_pt_br, text_pt_pt FROM ptbr_buf
                    """)
                    con.unregister("ptbr_buf")
                    n += len(buf)

                domain_total += n
                grand_total += n
                print(f"  [OK] {domain}/{split}: wrote {n:,} rows")

            print(f"[DOMAIN DONE] {domain}: wrote {domain_total:,} rows across {len(splits)} splits")

        final_count = con.execute("SELECT COUNT(*) FROM ptbrvarid").fetchone()[0]
        print(f"\n[ALL DONE] Inserted {grand_total:,} rows total. Table now has {final_count:,} rows.")

# ----- example usage -----
DB_PATH = "../data/duckdb/subs.duckdb"   # <-- your file
ingest_ptbrvarid_all_to_duckdb(DB_PATH, keep_accents=True, num_proc=1, batch_size_hf=1000, batch_size_db=5000)


[RESET] Cleared 0 old rows from ptbrvarid

[DOMAIN] journalistic – 1 splits
  [CLEAN] journalistic/train …


Map:   0%|          | 0/1842804 [00:00<?, ? examples/s]

Map:   0%|          | 0/1842804 [00:00<?, ? examples/s]

Map:   0%|          | 0/1842804 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1842804 [00:00<?, ? examples/s]

...

Filter:   0%|          | 0/133664 [00:00<?, ? examples/s]

Filter:   0%|          | 0/133552 [00:00<?, ? examples/s]

Map:   0%|          | 0/27969 [00:00<?, ? examples/s]

Filter:   0%|          | 0/27969 [00:00<?, ? examples/s]

  [OK] web/train: wrote 25,829 rows
[DOMAIN DONE] web: wrote 25,829 rows across 1 splits

[ALL DONE] Inserted 6,450,672 rows total. Table now has 6,450,672 rows.


In [4]:
import duckdb

DB_PATH = "../data/duckdb/subs.duckdb"

with duckdb.connect(str(DB_PATH)) as con:
    # See how many rows contain CR/LF/TAB before
    before = con.execute("""
      SELECT
        SUM(CASE WHEN text_pt_br IS NOT NULL AND regexp_matches(text_pt_br, '[\\r\\n\\t]') THEN 1 ELSE 0 END) AS br_hits,
        SUM(CASE WHEN text_pt_pt IS NOT NULL AND regexp_matches(text_pt_pt, '[\\r\\n\\t]') THEN 1 ELSE 0 END) AS pt_hits
      FROM ptbrvarid 
    """).fetchone()
    print("rows with CR/LF/TAB → BR/PT:", before)

    # Normalize: collapse ANY whitespace to a single space and trim ends
    con.execute("""
      UPDATE ptbrvarid
      SET
        text_pt_br = CASE
          WHEN text_pt_br IS NULL THEN NULL
          ELSE TRIM(regexp_replace(text_pt_br, '\\s+', ' '))
        END,
        text_pt_pt = CASE
          WHEN text_pt_pt IS NULL THEN NULL
          ELSE TRIM(regexp_replace(text_pt_pt, '\\s+', ' '))
        END
      WHERE
        (text_pt_br IS NOT NULL AND regexp_matches(text_pt_br, '[\\r\\n\\t]'))
        OR
        (text_pt_pt IS NOT NULL AND regexp_matches(text_pt_pt, '[\\r\\n\\t]'));
    """)

    con.execute("CHECKPOINT")

    after = con.execute("""
      SELECT
        SUM(CASE WHEN text_pt_br IS NOT NULL AND regexp_matches(text_pt_br, '[\\r\\n\\t]') THEN 1 ELSE 0 END),
        SUM(CASE WHEN text_pt_pt IS NOT NULL AND regexp_matches(text_pt_pt, '[\\r\\n\\t]') THEN 1 ELSE 0 END)
      FROM ptbrvarid
    """).fetchone()
    print("remaining rows with CR/LF/TAB → BR/PT:", after)


rows with CR/LF/TAB → BR/PT: (35455, 8578)
remaining rows with CR/LF/TAB → BR/PT: (35373, 8491)


In [4]:
con = duckdb.connect('../data/duckdb/subs.duckdb')
con.execute('SELECT * FROM ptbrvarid LIMIT 50000').df()

,dataset,split,label,text_pt_br,text_pt_pt,domain
0,PtBrVarId,train,0,None,"Cardoso e Cunha ""Insatisfatório"" O resultado do referendo francês é insatisfatório, tímido e modesto. Os problemas da Comunidade Europeia exigiam, da parte de um país que sempre esteve na linha da frente, uma vitória mais clara. A vitória tangencial do ""sim"" dá que pensar. Esta margem tão pequena que deu a vitória ao ""sim"" torna imprevisível o que vai acontecer na Grã-Bretanha.",journalistic
1,PtBrVarId,train,0,None,"Santo Tirso de fora Apesar da intenção das principais autarquias do Ave, para já Santo Tirso quer ficar de fora deste processo. Joaquim Couto, o presidente da edilidade tirsense, não está muito convencido da interesse das empresas e prefere concessionar alguns serviços a privados. É o caso da recolha e transporte dos lixos domésticos e da gestão e exploração do abastecimento de água. ""A concessão a privados dá mais transparência à gestão da autarquia"", referiu ao PÚBLICO o edil. Quanto às empresas públicas municipais, Couto duvida da sua utilidade e pergunta mesmo se valerá a pena autonomizar sem ter garantias de que não haverá um aumento das despesas e melhorias nos serviços. De resto, o autarca garante ainda que ""a gestão privada das coisas tem-se mostrado mais eficiente para os dinheiros públicos"". Em vez da legislação que permite a criação de empresas municipais, Couto preferia ter visto aprovada legislação que aumentasse o poder das câmaras e que lhes desse ""mais autonomia financeira"". ""Neste campo, somos os últimos da Europa"", refere, garantindo ainda que as verbas distribuídas pelo Estado às autarquias deveriam ""triplicar"". É que, diz, ""nenhuma empresa funciona se não tiver uma lei de financiamento capaz"". Emília Monteiro",journalistic
2,PtBrVarId,train,0,None,"""1 Rpm"" é a abreviatura para ""Uma Revolução por Minuto"", e a respectiva tradução inglesa, uma vez que os LX-90 de Rui Pregal da Cunha e Pedro Paulo Gonçalves (ex-Heróis do Mar) vão editar não um, mas dois álbuns de estreia. Lembra a estratégia dos seus colegas de editora Guns N' Roses, até porque o que permitirá diferenciar os dois discos nos escaparates serão os contrastes de o roxo numa das capas é amarelo na outra e vice-versa. Mas as comparações acabam aí: Ao contrário dos Roses, os discos dos LX-90 incluem as mesmas faixas, com a diferença de num disco elas surgirem cantadas em português e no outro em inglês. Como pode deduzir-se deste aparato, trata-se de um dos lançamentos mais ousados de sempre no capítulo da música feita no nosso país, e tem pelas próprias exigências que se vítima de uma série de acidentes de percurso. Embora o disco começasse por ser gravado nos estúdios Exit, com a produção dos ingleses Sam e Danny, estes consideraram que faltavam ali máquinas indispensáveis, e o grupo mudou-se para os estúdios de Paço de Arcos da Valentim de Carvalho. Em Setembro, com a mesma dupla de produtores, a banda voou para Londres, no intuito de ultimar as misturas finais, tendo-se então avançado o princípio deste mês como data de edição do trabalho.",journalistic
3,PtBrVarId,train,0,None,"Luís António Mendes Nobre pode, em Outubro, por alturada 5ª edição do Festival Internacional de Música de Macau, concretizar, como confessou, ""um velho sonho"". Do Oriente, Mendes Nobre só conhece a Tailândia, onde fez férias há dois anos. Para o pianista Adriano Jordão, director artístico do certame, o concurso ""correu muito bem. À Missão de Macau chegaram 306 boletins, o que para o nível exigido nas perguntas pode considerar-se bastante bom"".",journalistic
4,PtBrVarId,train,0,None,"Não posso dizer mais nada. Por que é que me está a fazer perguntas sobre Lisboa? Porque foi presidente da Câmara durante dez anos e nota-se que foi uma fase muito marcante da sua vida, à qual ficou afectivamente muito ligado.",journalistic
...,...,...,...,...,...,...
49995,PtBrVarId,train,0,None,"De algumas das tendências circunstanciais do ano que passou pode-se salientar o sucesso 

In [5]:
con.close()

**//FRMT DATASET**

In [8]:
GITHUB_RAW = ("https://raw.githubusercontent.com/google-research/google-research/"
              "HEAD/frmt/dataset")

buckets = {          # bucket → filename prefix inside that bucket
    "lexical": "pt_lexical",
    "entity" : "pt_entity",
    "random" : "pt_random",
}

splits   = ["dev", "test", "exemplars"]        # the paper’s three splits
regions  = ["pt-BR", "pt-PT"]                  # ⬅️ we ignore zh-* files

def urls(bucket):
    prefix = buckets[bucket]
    return [f"{GITHUB_RAW}/{bucket}_bucket/"
            f"{prefix}_{split}_en_{region}.tsv"
            for split in splits
            for region in regions]

# sanity-check
print(urls("entity")[:3])


['https://raw.githubusercontent.com/google-research/google-research/HEAD/frmt/dataset/entity_bucket/pt_entity_dev_en_pt-BR.tsv', 'https://raw.githubusercontent.com/google-research/google-research/HEAD/frmt/dataset/entity_bucket/pt_entity_dev_en_pt-PT.tsv', 'https://raw.githubusercontent.com/google-research/google-research/HEAD/frmt/dataset/entity_bucket/pt_entity_test_en_pt-BR.tsv']


In [9]:
def split_key(bucket, split, region):
    # pt-BR  ➜  pt_BR   (dash → underscore)
    return f"{bucket}_{split}_{region.replace('-', '_')}"

data_files = {
    split_key(bucket, split, region): [
        f"{GITHUB_RAW}/{bucket}_bucket/"
        f"{buckets[bucket]}_{split}_en_{region}.tsv"
    ]
    for bucket in buckets
    for split  in splits
    for region in regions
}

from datasets import load_dataset, DatasetDict

ds = load_dataset(
        "csv",
        data_files   = data_files,
        delimiter    = "\t",
        column_names = ["en", "pt"],
)

print(list(ds.keys())[:6])
# ['lexical_dev_pt_BR', 'lexical_dev_pt_PT', 'lexical_test_pt_BR', …]


['lexical_dev_pt_BR', 'lexical_dev_pt_PT', 'lexical_test_pt_BR', 'lexical_test_pt_PT', 'lexical_exemplars_pt_BR', 'lexical_exemplars_pt_PT']


In [10]:
entity_br = pd.DataFrame(ds['entity_dev_pt_BR'])

In [11]:
entity_br

,en,pt
0,"Constâncio was Secretary of State for Planning in the I and II Provisional Government of Portugal from 1974 to 1975, and Secretary of State for Budget and Planning in 1976 in the IV Provisional Government.","Constâncio foi Secretário de Estado para Planejamento no 1º e 2º Governos Provisórios de Portugal, de 1974 a 1975, e Secretário de Estado para Orçamento e Planejamento em 1976 no 4º Governo Provisório."
1,"He then became Finance Minister from January to August 1978 in the II Constitutional Government of Portugal, and is therefore until now the youngest Portuguese Finance Minister since the revolution.","Depois, tornou-se Ministro das Finanças de janeiro a agosto de 1978 no 2º Governo Constitucional de Portugal, e é, portanto, até hoje, o Ministro das Finanças português mais jovem desde a revolução."
2,Constâncio was secretary-general of the Socialist Party from 1986 to 1989.,"De 1986 a 1989, Constâncio foi secretário-geral do Partido Socialista."
3,"He lost the legislative elections of 19 July 1987, but remained in office.","Ele perdeu as eleições legislativas de 19 de julho de 1987, mas permaneceu no gabinete."
4,"He resigned the following year, being replaced by Jorge Sampaio.","No ano seguinte, ele renunciou, sendo substituído por Jorge Sampaio."
...,...,...
930,"The northern and western sides of the castle, on the other hand, were naturally protected by the steep hillside sloping downward from the castle's foundations.","Os lados norte e oeste do castelo, por sua vez, eram naturalmente protegidos pela colina íngreme inclinada para baixo das fundações do castelo."
931,"The castle is also partially encircled by a moat, now dry.","O castelo também é, em parte, rodeado por um fosso, agora seco."
932,The main entrance is fronted by a stone bridge across the moat.,"De frente para a entrada principal, há uma ponte de pedra sobre o fosso."
933,"On the west side, there is a long curtain wall extending downhill, ending at a tower (the Torre De Couraça).","No lado oeste, há uma divisória que se estende para baixo, terminando em uma torre (a Torre de Couraça)."


In [12]:
entity_pt = pd.DataFrame(ds['entity_dev_pt_PT'])
entity_pt[entity_pt["en"].duplicated(keep=False)].head()

,en,pt
52,"After the Portuguese legislative election of 2009, held on 27 September 2009, José Sócrates was elected for a second term as Prime Minister of Portugal.","Após as eleições legislativas portuguesas de 2009, realizadas em 27 de setembro de 2009, José Sócrates foi eleito pela segunda vez Primeiro-Ministro de Portugal."
64,"After the Portuguese legislative election of 2009, held on 27 September 2009, José Sócrates was elected for a second term as Prime Minister of Portugal.","Após as eleições legislativas portuguesas de 2009, realizadas em 27 de setembro de 2009, José Sócrates foi eleito para um segundo mandato como Primeiro-Ministro de Portugal."


In [13]:
entity_br[entity_br["en"].duplicated(keep=False)].head()

,en,pt
52,"After the Portuguese legislative election of 2009, held on 27 September 2009, José Sócrates was elected for a second term as Prime Minister of Portugal.","Após a eleição legislativa portuguesa de 2009, realizada em 27 de setembro de 2009, José Sócrates foi eleito para um segundo mandato como primeiro-ministro de Portugal."
64,"After the Portuguese legislative election of 2009, held on 27 September 2009, José Sócrates was elected for a second term as Prime Minister of Portugal.","Após a eleição legislativa portuguesa de 2009, realizada em 27 de setembro de 2009, José Sócrates foi eleito para um segundo mandato como primeiro-ministro de Portugal."


**//GOLD COLLECTION**

In [14]:
ds = load_dataset("joaosanches/golden_collection")

ds


DatasetDict({
    gold_collection: Dataset({
        features: ['text'],
        num_rows: 500
    })
    referencia_DeepL: Dataset({
        features: ['text'],
        num_rows: 500
    })
    referencia_manual: Dataset({
        features: ['text'],
        num_rows: 500
    })
})

In [15]:
pd.DataFrame(ds['gold_collection'])

,text
0,"Segundo Kellner, apesar de o animal ser um baixinho (poderia atingir, no máximo, 2,5 metros de altura), suas patas e bacia têm características anatômicas muito semelhantes às do ilustre réptil norte-americano.\n"
1,"Para a ONG, há evidências de que as companhias que mais exportam madeira para os EUA estejam envolvidas com o comércio ilegal do produto.\n"
2,"Mas, segundo a agência de notícias France Presse, a Venezuela já tem um caso suspeito: um homem de 48 anos que viajou recentemente à China.\n"
3,Ele afirmou que dieta e exercícios devem continuar a protagonizar tratamentos para emagrecer.\n
4,"O biólogo William Eberhard, da Universidade da Costa Rica, descobriu que as larvas desse inseto, ao parasitar a aranha Plesiometa argyra, provocam mudanças no comportamento da hospedeira.\n"
...,...
495,"Isso significa que a preposição é o termo que relaciona substantivo a substantivo, verbo a substantivo, substantivo a verbo, adjetivo a substantivo, advérbio a substantivo, etc.\n"
496,Foi então que a vestimenta mais feminina que se conhece começou a ganhar forma: o espartilho.\n
497,Um de seus professores foi Martin Wegelius.\n
498,"Nessa época, iniciou uma verdadeira polêmica com o escritor democrata Björnstjerne Björnson, através de correspondência.\n"


In [16]:
pd.DataFrame(ds['referencia_manual'])

,text
0,"Segundo Kellner, apesar de o animal ser muito pequeno (poderia atingir, no máximo, 2,5 metros de altura), as suas patas e bacia têm características anatómicas muito semelhantes às do ilustre réptil norte-americano.\n"
1,"Para a ONG, há evidências de que as empresas que mais exportam madeira para os EUA estejam envolvidas com o comércio ilegal do produto.\n"
2,"Mas, segundo a agência de notícias France Presse, a Venezuela já tem um caso suspeito: um homem de 48 anos que viajou recentemente à China.\n"
3,Ele afirmou que dieta e exercícios devem continuar a protagonizar tratamentos para emagrecer.\n
4,"O biólogo William Eberhard, da Universidade da Costa Rica, descobriu que as larvas desse inseto, ao parasitar a aranha Plesiometa argyra, provocam mudanças no comportamento da hospedeira.\n"
...,...
495,"Isso significa que a preposição é o termo que relaciona substantivo a substantivo, verbo a substantivo, substantivo a verbo, adjetivo a substantivo, advérbio a substantivo, etc.\n"
496,Foi então que a peça de vestuário mais feminina que se conhece começou a ganhar forma: o espartilho.\n
497,Um dos seus professores foi Martin Wegelius.\n
498,"Nessa época, iniciou-se uma verdadeira polémica com o escritor democrata Björnstjerne Björnson, através de correspondência.\n"
